In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import string
from tqdm import tqdm

In [2]:
def thongke(sentence,tu_khoa,tu_di_kem,tu_cam):
    sentence = str(sentence).translate(str.maketrans('', '', string.punctuation)).lower()
    K = np.zeros(len(tu_khoa))
    cam = False
    ok = False
    for word in tu_cam:
        if word in sentence:
            cam = True
            break
    if cam:
        return K
    for word in tu_di_kem:
        if word in sentence:
            ok = True
            break
    if not ok:
        return K
    for i in range(len(tu_khoa)):
        if tu_khoa[i] in sentence:
            K[i] = 1
    return K


In [3]:
def thongkecactinh(tinh, all_tinh, tu_khoa, tu_di_kem, tu_cam):
    PATH = f"Bo_4T/tinh/{all_tinh.loc[tinh,'file_name']}.csv"
    df_tinh = pd.read_csv(PATH)
    content = df_tinh.iloc[:,5]
    thongketinh = []
    for i in range(len(content)):
        K = thongke(content[i],tu_khoa,tu_di_kem,tu_cam)
        thongketinh.append(K)
    result = np.sum(np.stack(thongketinh,axis = 0),axis = 0)
    return result
    

In [4]:
def thongkecanuoc(tinh:pd.DataFrame, tu_khoa, tu_di_kem, tu_cam):
    cac_tinh = tinh.index
    full_Vietnam = []
    for i in tqdm(range(len(cac_tinh))):
        result = thongkecactinh(cac_tinh[i], tinh, tu_khoa, tu_di_kem, tu_cam)
        full_Vietnam.append(result)
    ketqua = np.stack(full_Vietnam,axis = 0)
    return ketqua

In [5]:
all_tinh = pd.read_csv('Bo_4T/tinh.csv',index_col='tinh')

In [8]:
for i in range(2,6):
    a_words = pd.read_csv(f'Bo_4T/add_word/n{i}.csv')
    tu_di_kem = a_words.keyword.values
    b_words = pd.read_csv(f'Bo_4T/stop_word/n{i}.csv')
    tu_cam = b_words.keyword.values
    keywords = pd.read_csv('Bo_4T/keyword.csv')
    tu_khoa = keywords.keyword.values
    tu_di_kem = list(map(lambda x:x.lower(),tu_di_kem))
    tu_cam = list(map(lambda x:x.lower(),tu_cam))
    tu_khoa = list(map(lambda x:x.lower(),tu_khoa))
    ketqua = thongkecanuoc(all_tinh, tu_khoa, tu_di_kem, tu_cam)
    df_ketqua = pd.DataFrame(data = ketqua, 
                  index = all_tinh.index, 
                  columns = tu_khoa).astype(int)
    df_ketqua.to_csv(f'Bo_4T/result/ketqua{i}.csv')

100%|██████████| 61/61 [08:18<00:00,  8.17s/it]
